In [ ]:
import keras
from keras.datasets import cifar10

## cifar 10 데이터 로드
(x_train , y_train),(x_test, y_test)= cifar10.load_data()

import matplotlib.pyplot as plt
import numpy as np


## cifar 10 dataset 시각화
fig = plt.figure(figsize=(20,5))
for i in range(36):
  ax = fig.add_subplot(3,12, i+1, xticks=[], yticks=[])
  ax.imshow(np.squeeze(x_train[i]))


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

from keras.utils import to_categorical
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

## normailze 대신 standarized 함 normalize = x_train / 255
## 각각의 rgb를 합쳐준다
mean = np.mean(x_train, axis=(0,1,2,3))
std = np.std(x_train, axis=(0,1,2,3))
## 데이터 정규화 (픽셀값 - 픽셀값의 평균) / 표준편차
x_train =(x_train-mean) / (std+1e-7)
x_valid =(x_valid-mean) / (std+1e-7)
x_test =(x_test-mean) / (std+1e-7)

## one-hot encoding
## cifar10 / class = 10 / 라벨이 10개
num_classes =  len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid , num_classes)
y_test = to_categorical(y_test, num_classes)

print(x_train.shape,  x_valid.shape, x_test.shape)


In [ ]:


from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout ,BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

## 데이터 강화 -> 과적합 방지 및 모델 쫀쫀
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range= 0.1,
    height_shift_range= 0.1,
    horizontal_flip= True,
    vertical_flip=False
)
## 변환 데이터 생성
datagen.fit(x_train)


In [ ]:

x_train.shape[1:]


In [ ]:

## 앞에서 스탠달다이즈 했기때문에 batch_noraml에서 non-trainable이 걸린다.
## kernel_regularizer -> 과적합 방지
from keras import regularizers
model = Sequential()
model.add(Conv2D(filters =32, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=x_train.shape[1:]  , activation='relu'))
## 이미 추출된 특징을 다시 정규화
model.add(BatchNormalization())
model.add(Conv2D(filters =32, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4)  , activation='relu'))
model.add(BatchNormalization())
## 이미지 특징들의 중요요소를 압축 -> 파라미터수 조절 -> 학습량 다운
model.add(MaxPooling2D(pool_size = 2))
## 뉴런들을 비활성화해서 가중치가 고루 분배되게 만듦 -> 과적합 방지
model.add(Dropout(0.2))
model.add(Conv2D(filters =64, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4)  , activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =64, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4)  , activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.3))
model.add(Conv2D(filters =128, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4)  , activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =128, kernel_size=3 , padding='same', kernel_regularizer=regularizers.l2(1e-4)  , activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
### 풀링을 거치면 입력 32*32*3 -> 4*4*128로 변경
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

from keras.mixed_precision.loss_scale_optimizer import optimizers
from keras.callbacks import ModelCheckpoint , EarlyStopping
from keras.optimizers import Adam
## BGD 데이터 전체가 1 epoch으로 --> 메모리 많이 씀
## SGD 데이터를 한개가 1 epoch  --> 느림 
## 미니배치 (BGD와 SGD의 중간)
batch_size = 128
epochs = 125

## load_weight으로 불러오기 위한 체크포인트, 에폭마다 모델저장
checkpoint = ModelCheckpoint('model.125epochs.hdf5', verbose=1 , save_best_only=True)
earlyStopping = EarlyStopping (monitor='val_loss', min_delta = 0, patience=10)
## learning rate / decay 감쇠율
optimizer = Adam(lr=0.0001, decay= 1e-6)

##categorical_crossentropy 정확도와 손실률은 꼭 정비례는 아니다.
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

## 강화데이터로 학습
hist = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), callbacks=[checkpoint], epochs= epochs, verbose=2, validation_data=(x_valid, y_valid))

scores = model.evaluate(x_valid,y_valid, batch_size=128, verbose=1)

print(f'loss : {scores[0]} , acc : {scores[1]}' )


In [ ]:

import matplotlib.pyplot as plt

## history['loss'] 손실율 / history['acc'] 정확도
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='test')
plt.legend()
plt.show()


In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout ,BatchNormalization , AveragePooling2D
model2 = Sequential()
model2.add(Conv2D(6, kernel_size=5 , strides=1 ,activation='tanh', input_shape=(32,32,3), padding='same' ))
model2.add(AveragePooling2D(pool_size=2, strides=2 ))
model2.add(Conv2D(16, kernel_size=5 , strides=1 ,activation='tanh' ))
model2.add(AveragePooling2D(pool_size=2, strides=2 ))
model2.add(Conv2D(120, kernel_size=5 , strides=1 ,activation='tanh' ))
model2.add(Flatten())
#model2.add(Dense(120, activation='tanh'))
model2.add(Dense(84, activation='tanh'))
model2.add(Dense(10, activation='softmax'))
model2.summary()


from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.datasets import cifar10
(x_train , y_train),(x_test, y_test)= cifar10.load_data()
## valid 층을 train에서 따로 떼어 놓음 
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

## 데이터 정규화
x_train = x_train.astype('float32')/ 255
x_test = x_test.astype('float32')/ 255
x_valid=x_valid.astype('float32')/255

from keras.utils import to_categorical


## one-hot encoding
## cifar10 / class = 10 / 라벨이 10개
num_classes =  len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_valid = to_categorical(y_valid, num_classes)

print(x_train.shape,  x_valid.shape, x_test.shape)

## 에포크 수에 따라 lr 조절
def schedule(epoch):
  if epoch <=2 :
    lr = 0.0005
  elif epoch >2 and epoch <=5:
    lr = 0.0002
  elif epoch >5 and epoch <9 :
    lr = 0.00005
  else : 
    lr =0.00001
  return lr 

## 스케줄러를 통한 lr 조절(선행 함수 필요)
lr_scheduler = LearningRateScheduler(schedule)
checkpoint = ModelCheckpoint('model.125epochs.hdf5', verbose=1 ,monitor= 'val_accuracy', save_best_only=True)
callboacks= [checkpoint, lr_scheduler]

model2.compile(loss='categorical_crossentropy', optimizer = 'sgd', metrics=['accuracy'])
hist = model2.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), callbacks= callboacks, verbose=2 , shuffle=True)

model2.load_weights('/content/model.relu.125epochs.hdf5')
model2.evaluate(x_test,y_test, verbose=2)


In [ ]:

from keras.regularizers import l2

alexnet= Sequential()
alexnet.add(Conv2D(64, kernel_size=7, activation='relu', input_shape=(32,32,3)))
#alexnet.add(MaxPooling2D(pool_size=2, strides=2 ))
alexnet.add(BatchNormalization())
alexnet.add(Conv2D(192, kernel_size=5, activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
alexnet.add(MaxPooling2D(pool_size=3, strides=2 ))
alexnet.add(BatchNormalization())
alexnet.add(Conv2D(256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
alexnet.add(Conv2D(256, kernel_size=3, activation='relu' ,padding='same', kernel_regularizer=l2(0.0005)))
alexnet.add(Conv2D(256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
alexnet.add(MaxPooling2D(pool_size=3, strides=2 ))

alexnet.add(Flatten())
alexnet.add(Dense(2048, activation='relu'))
alexnet.add(Dropout(0.5))
alexnet.add(Dense(2048, activation='relu'))
alexnet.add(Dropout(0.5))
alexnet.add(Dense(10, activation='softmax'))
alexnet.summary()

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import numpy as np
from keras.optimizers import SGD
## 검증오차가 정체될 때마다 학습률 10프로 감소
reduce_lr = ReduceLROnPlateau(monitor ='val_loss', factor=np.sqrt(0.1))
optimizer = SGD(lr=0.01, momentum=0.9 )


from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.datasets import cifar10
(x_train , y_train),(x_test, y_test)= cifar10.load_data()
## valid 층을 train에서 따로 떼어 놓음 
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

## 데이터 정규화
x_train = x_train.astype('float32')/ 255
x_test = x_test.astype('float32')/ 255
x_valid=x_valid.astype('float32')/255

from keras.utils import to_categorical
## one-hot encoding
## cifar10 / class = 10 / 라벨이 10개
num_classes =  len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_valid = to_categorical(y_valid, num_classes)

print(x_train.shape,  x_valid.shape, x_test.shape)


alexnet.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
hist = alexnet.fit(x_train, y_train, batch_size=128, epochs=90, validation_data=(x_valid, y_valid), callbacks= [reduce_lr], verbose=2 , shuffle=True)